<a href="https://colab.research.google.com/github/Bkirop/Coursera_Capstone/blob/master/Hospital%20Management%20system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#Lists
list_a=[1,2,3,4]
list_b=[1, "Today", True, 3.0]
print(list_a)
print(list_a[2])

# Problem: Create a functions that shows sum of two numbers

def add_numbers(a, b):
  return (a + b)

a = 10
b = 5
print(add_numbers(a, b))

[1, 2, 3, 4]
3
15


# Task
Create a hospital management system in Python that manages drugs (stock, in-store, prescribed) and patients (in wards, assigned to beds).

## Understand the scope

### Subtask:
Clarify the key features and functionalities required for drug and patient management.


**Reasoning**:
Based on the task description and the overall plan, I need to list the key features and functionalities for drug and patient management. I will list them in the code block using markdown format to make it clear.



In [6]:
"""
Key Features and Functionalities for Hospital Management System:

Drug Management:
- Track drug stock levels (total quantity available).
- Manage in-store drug quantities (drugs physically present in the pharmacy/storage).
- Handle prescribed drugs (associate drugs with patients and track dispensing).
- Add new drugs to the system.
- Update drug information (e.g., stock, price).
- Remove drugs from the system (e.g., expired or discontinued).

Patient Management:
- Admit new patients to the hospital.
- Assign patients to specific wards.
- Assign patients to available beds within wards.
- Track patient location (current ward and bed).
- Discharge patients from the hospital.
- Update patient information.

Interaction between Drug and Patient Management:
- Link prescribed drugs to specific patients.
- Track which drugs have been administered to which patient.
- Manage drug dispensing based on prescriptions for patients.
"""

'\nKey Features and Functionalities for Hospital Management System:\n\nDrug Management:\n- Track drug stock levels (total quantity available).\n- Manage in-store drug quantities (drugs physically present in the pharmacy/storage).\n- Handle prescribed drugs (associate drugs with patients and track dispensing).\n- Add new drugs to the system.\n- Update drug information (e.g., stock, price).\n- Remove drugs from the system (e.g., expired or discontinued).\n\nPatient Management:\n- Admit new patients to the hospital.\n- Assign patients to specific wards.\n- Assign patients to available beds within wards.\n- Track patient location (current ward and bed).\n- Discharge patients from the hospital.\n- Update patient information.\n\nInteraction between Drug and Patient Management:\n- Link prescribed drugs to specific patients.\n- Track which drugs have been administered to which patient.\n- Manage drug dispensing based on prescriptions for patients.\n'

## Data modeling

### Subtask:
Design the data structures (e.g., tables, classes) to represent drugs, stock, prescriptions, patients, wards, and beds.


**Reasoning**:
Define the Python classes for the hospital management system entities as outlined in the instructions, including their attributes, relationships, and initialization methods.



In [7]:
class Drug:
    def __init__(self, drug_id, name, total_stock, in_store_stock):
        self.drug_id = drug_id
        self.name = name
        self.total_stock = total_stock
        self.in_store_stock = in_store_stock

class Stock:
    def __init__(self, drug, quantity):
        self.drug = drug  # Reference to a Drug instance
        self.quantity = quantity

class Prescription:
    def __init__(self, prescription_id, patient, drugs):
        self.prescription_id = prescription_id
        self.patient = patient  # Reference to a Patient instance
        self.drugs = drugs  # List of Drug instances

class Patient:
    def __init__(self, patient_id, name, assigned_ward=None, assigned_bed=None):
        self.patient_id = patient_id
        self.name = name
        self.assigned_ward = assigned_ward  # Reference to a Ward instance
        self.assigned_bed = assigned_bed  # Reference to a Bed instance
        self.prescriptions = [] # List of Prescription instances

class Ward:
    def __init__(self, ward_id, name, capacity):
        self.ward_id = ward_id
        self.name = name
        self.capacity = capacity
        self.patients = [] # List of Patient instances
        self.beds = [] # List of Bed instances

class Bed:
    def __init__(self, bed_id, ward, is_occupied=False):
        self.bed_id = bed_id
        self.ward = ward  # Reference to a Ward instance
        self.is_occupied = is_occupied

## Drug management implementation

### Subtask:
Write code to manage drug inventory (stock, in-store, prescribed), including functions for adding, removing, and updating drug information.


**Reasoning**:
Implement a class to manage drug inventory, including functions for adding, updating, prescribing, dispensing, and removing drugs, based on the data models defined in the previous step.



In [8]:
class DrugInventory:
    def __init__(self):
        self.drugs = {}  # Dictionary to store drugs, keyed by drug_id

    def add_drug(self, drug_id, name, total_stock, in_store_stock):
        if drug_id in self.drugs:
            print(f"Drug with ID {drug_id} already exists.")
            return False
        new_drug = Drug(drug_id, name, total_stock, in_store_stock)
        self.drugs[drug_id] = new_drug
        print(f"Drug '{name}' added with ID {drug_id}.")
        return True

    def update_drug_stock(self, drug_id, total_stock=None, in_store_stock=None):
        if drug_id not in self.drugs:
            print(f"Drug with ID {drug_id} not found.")
            return False
        drug = self.drugs[drug_id]
        if total_stock is not None:
            drug.total_stock = total_stock
        if in_store_stock is not None:
            drug.in_store_stock = in_store_stock
        print(f"Stock for drug '{drug.name}' updated.")
        return True

    def prescribe_drug(self, patient, drug_id, quantity):
        if drug_id not in self.drugs:
            print(f"Drug with ID {drug_id} not found in inventory.")
            return False
        drug = self.drugs[drug_id]
        if drug.in_store_stock < quantity:
            print(f"Not enough '{drug.name}' in store. Available: {drug.in_store_stock}, Requested: {quantity}")
            return False

        # Assuming a simple prescription object for now, can be expanded later
        prescription_item = {"drug": drug, "quantity": quantity, "dispensed": False}
        patient.prescriptions.append(prescription_item)
        print(f"{quantity} units of '{drug.name}' prescribed to patient {patient.patient_id}.")
        return True

    def dispense_drug(self, patient, drug_id, quantity_to_dispense):
        if drug_id not in self.drugs:
            print(f"Drug with ID {drug_id} not found in inventory.")
            return False

        drug = self.drugs[drug_id]
        dispensed_count = 0
        for prescription_item in patient.prescriptions:
            if prescription_item["drug"].drug_id == drug_id and not prescription_item["dispensed"]:
                dispensable_quantity = min(quantity_to_dispense - dispensed_count, prescription_item["quantity"])
                if drug.in_store_stock < dispensable_quantity:
                    print(f"Not enough '{drug.name}' in store to dispense fully. Available: {drug.in_store_stock}")
                    dispensable_quantity = drug.in_store_stock

                drug.in_store_stock -= dispensable_quantity
                prescription_item["quantity"] -= dispensable_quantity # Reduce the quantity in the prescription
                dispensed_count += dispensable_quantity

                if prescription_item["quantity"] == 0:
                    prescription_item["dispensed"] = True # Mark as fully dispensed

                if dispensed_count == quantity_to_dispense:
                    break # Stop dispensing once the requested quantity is reached

        if dispensed_count > 0:
            print(f"{dispensed_count} units of '{drug.name}' dispensed to patient {patient.patient_id}.")
            return True
        else:
            print(f"No undispensed prescription for drug ID {drug_id} found for patient {patient.patient_id} or insufficient stock.")
            return False


    def remove_drug(self, drug_id):
        if drug_id not in self.drugs:
            print(f"Drug with ID {drug_id} not found.")
            return False

        # Basic check: Could add checks for prescribed or in-stock drugs later
        del self.drugs[drug_id]
        print(f"Drug with ID {drug_id} removed.")
        return True

# Example Usage (for testing the DrugInventory class)
inventory = DrugInventory()
inventory.add_drug("D001", "Aspirin", 1000, 500)
inventory.add_drug("D002", "Ibuprofen", 500, 200)

inventory.update_drug_stock("D001", in_store_stock=450)

# Create a dummy patient for testing prescription and dispensing
dummy_patient = Patient("P001", "John Doe")

inventory.prescribe_drug(dummy_patient, "D001", 50)
inventory.prescribe_drug(dummy_patient, "D002", 20)

inventory.dispense_drug(dummy_patient, "D001", 30)
inventory.dispense_drug(dummy_patient, "D002", 25) # Should dispense only 20

inventory.remove_drug("D001")
inventory.remove_drug("D003") # Should show not found

Drug 'Aspirin' added with ID D001.
Drug 'Ibuprofen' added with ID D002.
Stock for drug 'Aspirin' updated.
50 units of 'Aspirin' prescribed to patient P001.
20 units of 'Ibuprofen' prescribed to patient P001.
30 units of 'Aspirin' dispensed to patient P001.
20 units of 'Ibuprofen' dispensed to patient P001.
Drug with ID D001 removed.
Drug with ID D003 not found.


False

## Patient and ward management implementation

### Subtask:
Write code to manage patient information, ward allocation, and bed assignments.


**Reasoning**:
Implement the Hospital class with methods for managing wards, beds, and patients as requested in the subtask instructions.



In [ ]:
class Hospital:
    def __init__(self):
        self.wards = {}  # Dictionary to store wards, keyed by ward_id
        self.patients = {} # Dictionary to store patients, keyed by patient_id

    def add_ward(self, ward_id, name, capacity):
        if ward_id in self.wards:
            print(f"Ward with ID {ward_id} already exists.")
            return False
        new_ward = Ward(ward_id, name, capacity)
        self.wards[ward_id] = new_ward
        # Create beds for the ward
        for i in range(capacity):
            bed_id = f"{ward_id}-bed-{i+1}"
            new_bed = Bed(bed_id, new_ward)
            new_ward.beds.append(new_bed)
        print(f"Ward '{name}' added with ID {ward_id} and {capacity} beds.")
        return True

    def add_bed(self, ward_id, bed_id):
        if ward_id not in self.wards:
            print(f"Ward with ID {ward_id} not found.")
            return False
        ward = self.wards[ward_id]
        for bed in ward.beds:
            if bed.bed_id == bed_id:
                print(f"Bed with ID {bed_id} already exists in Ward {ward_id}.")
                return False
        if len(ward.beds) >= ward.capacity:
            print(f"Ward {ward_id} is already at full capacity.")
            return False
        new_bed = Bed(bed_id, ward)
        ward.beds.append(new_bed)
        print(f"Bed '{bed_id}' added to Ward {ward_id}.")
        return True

    def admit_patient(self, patient_id, name, ward_id):
        if patient_id in self.patients:
            print(f"Patient with ID {patient_id} already exists.")
            return False
        if ward_id not in self.wards:
            print(f"Ward with ID {ward_id} not found.")
            return False

        ward = self.wards[ward_id]
        available_bed = self.find_available_bed(ward_id)

        if available_bed:
            new_patient = Patient(patient_id, name, assigned_ward=ward, assigned_bed=available_bed)
            self.patients[patient_id] = new_patient
            ward.patients.append(new_patient)
            available_bed.is_occupied = True
            print(f"Patient '{name}' admitted with ID {patient_id} to Ward '{ward.name}', Bed '{available_bed.bed_id}'.")
            return new_patient
        else:
            print(f"No available beds in Ward {ward.name}.")
            return None

    def discharge_patient(self, patient_id):
        if patient_id not in self.patients:
            print(f"Patient with ID {patient_id} not found.")
            return False

        patient = self.patients[patient_id]
        if patient.assigned_ward and patient.assigned_bed:
            patient.assigned_ward.patients.remove(patient)
            patient.assigned_bed.is_occupied = False
            print(f"Patient '{patient.name}' discharged from Ward '{patient.assigned_ward.name}', Bed '{patient.assigned_bed.bed_id}'.")
            patient.assigned_ward = None
            patient.assigned_bed = None
        else:
            print(f"Patient '{patient.name}' was not assigned to a ward or bed.")

        del self.patients[patient_id]
        print(f"Patient with ID {patient_id} removed from the system.")
        return True

    def find_patient_by_id(self, patient_id):
        return self.patients.get(patient_id)

    def find_available_bed(self, ward_id):
        if ward_id not in self.wards:
            print(f"Ward with ID {ward_id} not found.")
            return None
        ward = self.wards[ward_id]
        for bed in ward.beds:
            if not bed.is_occupied:
                return bed
        return None

# Example Usage (for testing the Hospital class patient management)
hospital = Hospital()

# Add wards
hospital.add_ward("W001", "General Ward", 5)
hospital.add_ward("W002", "ICU", 2)
hospital.add_ward("W001", "General Ward 2", 10) # Should show already exists

# Add beds (optional, beds are added with ward capacity)
hospital.add_bed("W001", "W001-bed-6") # Should show at full capacity if capacity was 5
hospital.add_bed("W003", "W003-bed-1") # Should show ward not found

# Admit patients
patient1 = hospital.admit_patient("P001", "Alice Smith", "W001")
patient2 = hospital.admit_patient("P002", "Bob Johnson", "W001")
patient3 = hospital.admit_patient("P003", "Charlie Brown", "W001")
patient4 = hospital.admit_patient("P004", "David Green", "W001")
patient5 = hospital.admit_patient("P005", "Eve Black", "W001")
patient6 = hospital.admit_patient("P006", "Frank White", "W001") # Should show no available beds

patient_icu_1 = hospital.admit_patient("P007", "Grace Blue", "W002")

# Find patients
found_patient = hospital.find_patient_by_id("P003")
if found_patient:
    print(f"Found patient: {found_patient.name}")
else:
    print("Patient not found.")

not_found_patient = hospital.find_patient_by_id("P999")
if not_found_patient:
     print(f"Found patient: {not_found_patient.name}")
else:
     print("Patient not found.")

# Find available beds
available_bed_ward1 = hospital.find_available_bed("W001")
if available_bed_ward1:
    print(f"Available bed in W001: {available_bed_ward1.bed_id}")
else:
    print("No available beds in W001.")

available_bed_ward2 = hospital.find_available_bed("W002")
if available_bed_ward2:
    print(f"Available bed in W002: {available_bed_ward2.bed_id}")
else:
    print("No available beds in W002.")


# Discharge patients
hospital.discharge_patient("P002")
hospital.discharge_patient("P999") # Should show patient not found

# Admit another patient to the ward with a discharged patient
patient7 = hospital.admit_patient("P008", "Hannah Purple", "W001")